In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def Categorize(data_preprocess, N_category):
    max1 = max(data_preprocess.max())
    min1 = min(data_preprocess.min())
    data_afterprocess = round((max1 - data_preprocess)/(max1-min1)*(N_category-1))*((max1-min1)/(N_category-1))+min1
    return data_afterprocess


In [3]:
import pandas as pd
import numpy as np


#import random
path_data = 'ROOM_6/HVAC_Data.txt'
path_label ='ROOM_6/HVAC_Label.txt'
path_reward = 'ROOM_6/HVAC_Reward.txt'
data_action_state = pd.read_table(path_data, sep = ",")
data_next_state_pre = pd.read_table(path_label, sep = ",")
data_reward = pd.read_table(path_reward, sep = ",")
# Separate Action, State, and Next Staet
# We remove the state AIR_VOLUMN, which was created for visualization
data_action_pre = data_action_state.filter(regex='action')
data_state_pre = data_action_state.filter(regex='state')
data_state_pre_dropped = data_state_pre.drop(data_state_pre.columns[6:12], axis = 1) 
data_next_state_pre_drop = data_next_state_pre.drop(data_next_state_pre.columns[6:12], axis = 1) 

N_room = 6; 

#We cannot have continuous data in action and state, so we need to divide them into N categories
N_action = 20;
# N_temp = 30;
data_action = Categorize(data_action_pre, N_action)
# data_state = Categorize(data_state_pre_dropped, N_temp)
# Validate: data_action['action: AIR[$r2]'].unique()
# Validate: data_state['states: TEMP[$r1]'].unique()

data_state = round(data_state_pre_dropped,1)
data_next_state = round(data_next_state_pre_drop,1)

# Merge dataset: [action, state, new state, reward]
data = pd.concat([data_action, data_state, data_next_state,data_reward], axis = 1)

#Create training, validation, and testing set
length = len(data)
#random.shuffle(data)
data_shuffle = data.reindex(np.random.permutation(data.index))
data_train = data_shuffle[:int(0.8*length)]
# data_validate = data_shuffle[int(0.5*length):int(0.8*length)]
data_testing = data_shuffle[int(0.8*length):]

In [4]:
# The original state data in input is [1,6] array, we will transform it into [N_room, N_temp] hot-code format
input_state = tf.placeholder(shape = [N_room,N_room], dtype = tf.float32)
Weight = tf.Variable(tf.random_uniform([N_room, N_action], 0, 0.01))
Q_out = tf.matmul(input_state, Weight) # All possible Q with each action in the current state
predict = tf.argmax(Q_out, 1) # Find optimal action to achieve the max Q in each row

Q_next = tf.placeholder(shape = [1, N_action], dtype = tf.float32)
loss = tf.reduce_sum(tf.square(Q_next-Q_out))
trainer = trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)


In [ ]:
def Transition(Action, State):
    # If the pair of Action and State is found in the training data, return the corresponding Next State and Rewards
    # Otherwise, find the action that maximize the reward in that state in the training set, and return the 
    # correspondingNext State, adn Rewards
    
    # Return State_Next, Rewards

In [ ]:
init = tf.initialize_all_variables()

#Set up learning parameters
num_episodes = 2000;
y = 0.99 #Discount Factor


# Create arraylist to record Rewards
rList = []

with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset the first observation in the new episodes
        s = [10]*N_room # The first row of the data
        j = 0 # j is the number of steps, and set the maximum as 99 steps
        
        # The Q-Network
        while j < 99: 
            j+= 1
            
            # Use Weight to calcualte all the Q, and choose the optimal action and record all Q with potential Action
            a,Q_all = sess.run([predict,Q_out],feed_dict={input_state:np.identity(N_room)*s}) 
            
            #if np.random.rand(1) < e:
            #   a = env.action_space.sample() # Exploration
            
            # Get the new state and reward from the environment
            s1, r = transition(a, s)
            
            # Obtain the next Q by feeding the new state through the network
            Q1 = sess.run(Q_out, feed_dict={input_state:np.identity(N_room)*s1})
            
            # Obtain Maximum of Q_new and set as our target
            Q1_max = np.max(Q1)
            target_Q = r + y*Q1_max #Bellmen Equation
            # Train our network using target and predicted Q value
            _,W1 = sess.run([updateModel, Weight],feed_dict={input_state:np.identity(N_room)*s,Q_next:target_Q})
            r_all += r # Accumulate reward
            s = s1 # Enter the next state
        rList.append(r_all)